# Principal Component Analysis and Singular Value Decomposition

- Bowen Li
- 2016/10/05

## Outline
- What is Principal Component Analysis (PCA)?
- What is Singular Value Decomposition (SVD)?
- PCA and SVD

## 1. What is PCA?

**Motivation:** Identify the most meaningful basis to re-express data which can maximize the **signal-to-noise ratio (SNR):** $\sigma^2_{signal} / \sigma^2_{noise}$.

<img src="./images/snr.png" alt="Drawing" style="width: 300px;"/>

- Minimize redundancy (noise), measured by the magnitude of the covariance.
- Maximize the signal, measured by the variance; large variance corresponds to interesting structure.

Let the observed data be

$$
\underset{(n \times p)}X = \{x_{ij}\} =
\begin{bmatrix}
x_{11} & \cdots & x_{1p} \\
\vdots &        & \vdots \\ 
x_{n1} & \cdots & x_{np}
\end{bmatrix}
= [x_1, \cdots, x_p]
$$

- each row represents an *example* measured on $p$-dimensional features,
- each column, $x_j$, represents *feature* with *zero means*, $E(x_j) = 0$, or *centered variables,* $x_j \leftarrow x_j - \overline x_j$,

$$
x_j = 
\begin{bmatrix}
x_{1j} \\
\vdots \\ 
x_{nj}
\end{bmatrix}
$$

**Covariance matrix:**

$$
\underset{(p \times p)}{C_X} = \frac{1}{n}X^{T}X 
= \frac{1}{n}
\begin{bmatrix}
x^T_{1} \\
\vdots \\ 
x^T_{p}
\end{bmatrix}
[x_1,...,x_p]
= \frac{1}{n}
\begin{bmatrix}
x^T_{1}x_1 & \cdots & x^T_{1}x_p \\
\vdots     &        & \vdots \\ 
x^T_{p}x_1 & \cdots & x^T_{p}x_p
\end{bmatrix}
$$

**Goal:** Find some transformation, $Y = f(X)$, of X such that covariance of $Y$ is a *diagonal* matrix,

$$
\underset{(p \times p)}{C_Y} = \frac{1}{n}Y^{T}Y
$$

There are many ways for diagonalizing $C_X$, PCA arguably select the easiest method.

**Assumptions for PCA:**
- Linear transformation: Let $\underset{p \times p}E = [e_1,...,e_p]$,

$$
Y = XE
$$

- Large variance have important structure.
- The principal components are orthogonal. 

The last assumption provides an intuitive simplification that makes PCA soluble with linear algebra decomposition techniques. 

**Computation for PCA:** Based on the above assumptions, define
- First principal component = linear combination $e^T_1X$ that maximizes $Var(e^T_1X)$, subject to $e^T_1e_1 = 1$.
- Second principal component = linear combination $e^T_2X$ that maximizes $Var(e^T_2X)$, subject to $e^T_2e_2 = 1$ and $Cov(e^T_1X, e^T_2X) = 0$.
- At the ith step, ith principal component = linear combination $e^T_iX$ that maximizes $Var(e^T_iX)$, subject to $e^T_ie_i = 1$ and $Cov(e^T_kX, e^T_iX) = 0$ for $k < i$.

**Eigenvector decomposition (also called spectral decomposition):**
- Suppose $Y = XE$,

$$
C_Y = \frac{1}{n}Y^TY = \frac{1}{n}(XE)^T(XE) = E^T \left(\frac{1}{n}X^TX \right)E
$$

- Since $\frac{1}{n}X^TX$ is a *symmetric* matrix, it can be **diagonalized by a matrix of orthonormal eigenvectors** using **spectral decomposistion.** That is,

$$
\frac{1}{n}X^TX = VDV^T
$$

where $D = diag(d_1,...,d_p)$ is a *diagonal* matrix with rank-ordered set of **eigenvalues**, $d_1 \ge d_2 \ge ... \ge d_p$, and $V = [v_1,...,v_p]$ is the matrix of the corresponding **eigenvectors** with $V^TV=VV^T=1$.

- *Sketch of proof:* From the spectral decomposition, for all $i$, we have 

$$
\frac{1}{n}X^T X v_i = d_i v_i
$$

Then, $\frac{1}{n}X^T X V = V D \Rightarrow \frac{1}{n}X^T X = V D V^T$.

- Trick: Select $E = V$. Then since $V^TV=VV^T = I$, $C_Y$ now is a diagonal matrix

$$
C_Y = E^T \left( \frac{1}{n}X^T X \right) E
    = V^T \left( V D V^T \right) V = D
$$

**Summary of PCA:**
- Substract off the mean of each feature: $x_j \leftarrow x_j - \overline x_j$
- Compute the eigenvectors of covariance matrix, $\frac{1}{n}X^TX$
- Principal components of $X$: eigenvectors, $v_i$, of covariance matrix
- The ith diagonal value of $C_Y$: variance of $X$ along $v_i$ 

## 2. What is SVD?
First recall the eigenvector decomposition: $\frac{1}{n}X^T X v_i = d_i v_i$. Define
- **Singular values:** $\lambda_i = \sqrt{d_i}$, for $i = 1,...,k$
- Matrix $U = [ u_1,...,u_2 ]$, with $u_i$ defined by

$$
u_i = \frac{1}{\lambda_i}\left( \frac{1}{\sqrt{n}}X \right) v_i
$$

Then (1) $U$ is *orthonormal* matrix:
$u_i^T u_j = 
\begin{cases} 
1, \text{if } i =j \\
0, \text{otherwise}
\end{cases}
$
and (2) $\lVert X v_i\rVert = \lambda_i$

- *Sketch of proof:* $u_i^T u_j = \frac{1}{n} \left( \frac{1}{\lambda_i} X v_i \right)^T \left( \frac{1}{\lambda_j} X v_j \right) = \frac{1}{n} \frac{1}{\lambda_i \lambda_j} v_i^T X^T X v_j = \frac{1}{\lambda_i \lambda_j} v_i^T d_i v_j = \frac{\lambda_i}{\lambda_j} v_i^T v_j$. 
The first result follows. Further, we can obtain the second result similarly.

By rewriting we have

$$
\left( \frac{1}{\sqrt{n}}X \right) v_i = \lambda_i u_i
$$

That is, **normalized $X$ multiplied by an eigenvector of covariance of $X$, $\frac{1}{n} X^T X$, is equal to a scalar times another vector.**

- By constructing a new *diagonal* matrix of singular values, $\Sigma = diag(\lambda_1,...,\lambda_p)$, where $\lambda_1 \ge \lambda_2 \ge ... \ge \lambda_p$.
- Define $V = [v_1,...,v_p]$ and $U = [u_1,...,u_p]$.

**Matrix version of SVD:**

$$
\left( \frac{1}{\sqrt{n}}X \right) V = U \Sigma,
$$
$$
\frac{1}{\sqrt{n}}X = U \Sigma V^T
$$


Hence, **any arbitrary matrix $\frac{1}{\sqrt{n}}X$ can be converted to**
- **an orthogonal matrix (rotation): $U$**
- **a diagonal matrix (stretch): $\Sigma$**
- **another orthogonal matrix (second rotation): $V$**

## 3. PCA and SVD
From SVD, we have

$$
\frac{1}{n} X^T X = \left( \frac{1}{\sqrt{n}}X \right)^T \left( \frac{1}{\sqrt{n}}X \right)
                  = \left( U \Sigma V^T \right)^T \left( U \Sigma V^T \right) \\
                  = \left( V \Sigma U^T U \Sigma V^T \right)
                  = V \Sigma^2 V^T \equiv V D V^T
$$

That is, **squared singular value is equal to variance of $X$ along $v_i$, $\lambda_i^2 = d_i$.**

**How many pricipal components we shall use?**
- **Total variances:** $\textstyle \sum_{k=1}^p \lambda_k^2$.
- **Scree plot:** Choose the number of principal component by *elbow method.*

<img src="./images/scree_plot_pca.png" alt="Drawing" style="width: 300px;"/>

**More on SVD:**
- Recall from SVD, $\frac{1}{\sqrt{n}}X = U \Sigma V^T$
- Let $s < p = rank(X)$. Then the **reduced rank-$s$ least squares approximation to $\frac{1}{\sqrt{n}} X$ is**

$$
\frac{1}{\sqrt{n}}\widehat{X} = \sum_{k=1}^s \lambda_i u_i v_i^T
$$

which **minimizes**

$$
\frac{1}{n} \sum_{i=1}^n \sum_{j=1}^p \left( x_{ij} - \widehat{x}_{ij} \right)^2
= tr \left[ \left(\frac{1}{\sqrt{n}} (X - \widehat{X}) \right) \left(\frac{1}{\sqrt{n}} (X - \widehat{X}) \right)^T \right]
$$

over all matrices $\widehat{X}$ having rank no greater than $s$.

## References
- Johnson & Wichern (2002). Applied Multivariate Statistical Analysis.
- Shlens (arXiv, 2014). A Tutorial on Principal Component Analysis.